In [ ]:
// The Nature of Code
// Daniel Shiffman
// http://natureofcode.com

// Genetic Algorithm, Evolving Shakespeare

// A class to describe a population of virtual organisms
// In this case, each organism is just an instance of a DNA object

class Population {
  constructor(p, m, num) {

    this.population; // Array to hold the current population
    this.matingPool; // ArrayList which we will use for our "mating pool"
    this.generations = 0; // Number of generations
    this.finished = false; // Are we finished evolving?
    this.target = p; // Target phrase
    this.mutationRate = m; // Mutation rate
    this.perfectScore = 1;

    this.best = "";

    this.population = [];
    for (let i = 0; i < num; i++) {
      this.population[i] = new DNA(this.target.length);
    }
    this.matingPool = [];
    this.calcFitness();
  }

  // Fill our fitness array with a value for every member of the population
  calcFitness() {
    for (let i = 0; i < this.population.length; i++) {
      this.population[i].calcFitness(target);
    }
  }

  // Generate a mating pool
  naturalSelection() {
    // Clear the ArrayList
    this.matingPool = [];

    let maxFitness = 0;
    for (let i = 0; i < this.population.length; i++) {
      if (this.population[i].fitness > maxFitness) {
        maxFitness = this.population[i].fitness;
      }
    }

    // Based on fitness, each member will get added to the mating pool a certain number of times
    // a higher fitness = more entries to mating pool = more likely to be picked as a parent
    // a lower fitness = fewer entries to mating pool = less likely to be picked as a parent
    for (let i = 0; i < this.population.length; i++) {

      let fitness = map(this.population[i].fitness, 0, maxFitness, 0, 1);
      let n = floor(fitness * 100); // Arbitrary multiplier, we can also use monte carlo method
      for (let j = 0; j < n; j++) { // and pick two random numbers
        this.matingPool.push(this.population[i]);
      }
    }
  }

  // Create a new generation
  generate() {
    // Refill the population with children from the mating pool
    for (let i = 0; i < this.population.length; i++) {
      let a = floor(random(this.matingPool.length));
      let b = floor(random(this.matingPool.length));
      let partnerA = this.matingPool[a];
      let partnerB = this.matingPool[b];
      let child = partnerA.crossover(partnerB);
      child.mutate(this.mutationRate);
      this.population[i] = child;
    }
    this.generations++;
  }


  getBest() {
    return this.best;
  }

  // Compute the current "most fit" member of the population
  evaluate() {
    let worldrecord = 0.0;
    let index = 0;
    for (let i = 0; i < this.population.length; i++) {
      if (this.population[i].fitness > worldrecord) {
        index = i;
        worldrecord = this.population[i].fitness;
      }
    }

    this.best = this.population[index].getPhrase();
    if (worldrecord === this.perfectScore) {
      this.finished = true;
    }
  }

  isFinished() {
    return this.finished;
  }

  getGenerations() {
    return this.generations;
  }

  // Compute average fitness for the population
  getAverageFitness() {
    let total = 0;
    for (let i = 0; i < this.population.length; i++) {
      total += this.population[i].fitness;
    }
    return total / (this.population.length);
  }

  allPhrases() {
    let everything = "";

    let displayLimit = min(this.population.length, 50);


    for (let i = 0; i < displayLimit; i++) {
      everything += this.population[i].getPhrase() + "<br>";
    }
    return everything;
  }
}

# Python

In [159]:
import importlib
import numpy as np
import time

In [196]:
from dna import DNA # To get this to work, needed to put 'if name == main' at bottom of dna.py 

In [481]:
importlib.reload(dna)

<module 'dna' from '/home/mjmrose/workspace/sbox-mjmrose/Bids/genAlgo/dna.py'>

In [482]:
# A class to describe a population of virtual organisms
# In this case, each organism is just an instance of a DNA object

class Population:
    def __init__(self, tgt, mut_rate, pop_sz, fitness_exp):

        self.population = [] # Array to hold the current population
        self.mating_pool = [] # List which we will use for our "mating pool"
        self.generations = 0 # Number of generations
        self.finished = False # Are we finished evolving?
        self.tgt = tgt # Target phrase
        self.mut_rate = mut_rate # Mutation rate
        self.perfect_score = 1
        self.best = ""
        self.fitness_sum = 0
        self.fitness_exp = fitness_exp
        self.verbose = False
        self.debug = False

        for i in range(pop_sz):
            self.population.append( DNA( len(self.tgt) ) )

    # Fill our fitness array with a value for every member of the population
    def calc_fitness(self):
        if self.debug: print("start of calc_fitness")
        self.fitness_sum = 0
    
        for i in range( len(self.population) ):
            self.population[i].calc_fitness(self.tgt)
#             print(f"pre exp: {self.population[i].fitness}")
            self.population[i].fitness = self.population[i].fitness**self.fitness_exp
            self.fitness_sum += self.population[i].fitness

    def gen_mating_pool(self):
        if self.debug: print("start of gen_mating_pool")
        self.mating_pool = []
    
        for i in range( len(self.population) ): 
    
            # Appending (idx, normalized fitness) for each mem of pop
            self.mating_pool.append( (i, self.population[i].fitness / max(self.fitness_sum, 1e-4) ) )
    
        # Sorting by fitness score in descending order
        self.mating_pool.sort(reverse=True, key = lambda x : x[1])
        
    
    def pick_mem_from_mating_pool(self):
        '''
        Selects a member (mem) from the mating pool to participate in crossover.

        Steps for selection:
            1). Draw random number b/w 0-1 (val)
            2). Subtract normalized fitness of 1st mem of mating pool (highest fitness) from val
            3). If val is now negative, mem of pop corresponding to first mem of mating pool is selected for crossover.
            4). If val is still positive, move to 2nd mem of mating pool and repeat until val is negative
        '''
        if self.debug: print("Start of pick_mem_from_mating_pool")
            
        val = np.random.random()
        
        if self.verbose: print(f"val: {val}")
        if self.verbose: print(f"mating pool: {self.mating_pool}")
            
        for i in range( len(self.mating_pool) ):
            val -= self.mating_pool[i][1]
            if val < 0:
                break
        if self.verbose: print(f"idx of mating pool: {self.mating_pool[i][0]}")
        return self.mating_pool[i][0]
    

    # Create a new generation -->  Refill the population with children from the mating pool
    def gen_new_pop(self, num_new_mems, replace_bool):
        '''
        Generates new members (mems) of population by probalistically mating existing mems of the population.
        Mems of the population w/ higher fitness are more likely to be selected for mating.

        Inputs:
            num_new_mems: number of new mems to add to the population or to replace in existing population
            replace_bool: if True, will replace 'num_new_mems' mems of pop w/ lowest fitness w/ new children 
        '''
        if self.debug: print("Start of gen_new_pop")
        children = []
        
        for i in range( num_new_mems ): 
                
            idx1 = self.pick_mem_from_mating_pool()
            idx2 = self.pick_mem_from_mating_pool()
            
            partnerA = self.population[idx1]
            partnerB = self.population[idx2]
            
            if self.verbose: print(f"PartnerA: {partnerA.get_phrase()}")
            if self.verbose: print(f"PartnerB: {partnerB.get_phrase()}")
    
            child = partnerA.crossover(partnerB)
            child.mutate(self.mut_rate)
            if self.verbose: print(f"Child: {child.get_phrase()}")
                
            children.append(child)
    
#             if replace_bool:    
#                 noise = min(len(self.population) - 3, max(int(np.random.random() * np.random.randint(len(self.population))) - 1, 1) )
#                 if self.verbose: print(f"noise: {noise}")
#                 replace_idx = self.mating_pool[len(self.mating_pool) - noise][0]
#                 self.population[replace_idx] = child

        if replace_bool:    
            for i in range(len(children)):
                replace_idx = self.mating_pool[len(self.mating_pool) - i - 1][0]
                self.population[replace_idx] = children[i]
        else:
            self.population.extend(children)

        self.generations += 1


    def get_best(self):
        return self.best

    # Compute the current "most fit" member of the population
    def evaluate(self):
        world_record = 0
        idx = 0
        for i in range(len(self.population)): 
            if self.population[i].fitness > world_record:
                idx = i
                world_record = self.population[i].fitness
                
        print(f"World Record: {world_record**(1/self.fitness_exp)}")

        self.best = self.population[idx].get_phrase()
        if world_record**(1/self.fitness_exp) == self.perfect_score:
            self.finished = True

    def is_finished(self):
        return self.finished

    def get_generations(self):
        return self.generations

    # Compute average fitness for the population
    def get_average_fitness(self):
        total = 0
        for i in range( len( self.population ) ):
            total += self.population[i].fitness**(1/self.fitness_exp)
        return total / len(self.population)

In [483]:
def setup(tgt, pop_sz, mut_rate, fitness_exp):

    # Create a population with a target phrase, mutation rate, and population max
    population = Population(tgt, mut_rate, pop_sz, fitness_exp)
    
    return population    

In [512]:
def draw(population, num_new_mems, replace_bool):
    
    # Calculate fitness for each mem of pop, take fitness**fitness_exp and calc fitness sum
    population.calc_fitness()
    
    # Generate mating pool array by sorting normalized fitness values
    population.gen_mating_pool()
    
    # Generate new population mems by crossover b/w existing mems of mating pool
    # Either replace existing mems or add new mems to pop
    population.gen_new_pop(num_new_mems, replace_bool)
    
    # Compute most fit mem of pop and determine if finished
    population.evaluate()
    
    print(f"Best: {population.get_best()}")
    print(f"Average: {population.get_average_fitness()}")

    # If we found the target phrase, stop
    if population.is_finished():
        print("We did it :)")
        print(f"Result: {pop.get_best()}")
        print(f"Num gens: {pop.get_generations()}")

In [522]:
tgt = "To be or not to be."
pop_sz = 500
mut_rate = 0.3
fitness_exp = 2
pop = setup(tgt, pop_sz, mut_rate, fitness_exp)

# Testing speed of convergence

In [523]:
start = time.time()
for i in range(1000):
    draw(pop, 200, False)
    if pop.is_finished():
        break
print(f"\nTime: {time.time() - start}")

World Record: 0.10526315789473684
Best: tQ<bLP9*'c8I EqstcL
Average: 0.007593984962406012
World Record: 0.15789473684210525
Best: 0W+'QJ#c!n.telKIbua
Average: 0.017543859649122757
World Record: 0.21052631578947367
Best: Bo:se,\ua&onl.% h7P
Average: 0.025980861244019184
World Record: 0.21052631578947367
Best: Bo:se,\ua&onl.% h7P
Average: 0.033522267206477954
World Record: 0.21052631578947367
Best: Bo:se,\ua&onl.% h7P
Average: 0.040385964912281046
World Record: 0.2631578947368421
Best: &o99e1pOGnOtxtQQ.7"
Average: 0.045572755417957196
World Record: 0.2631578947368421
Best: &o99e1pOGnOtxtQQ.7"
Average: 0.05047091412742457
World Record: 0.3157894736842105
Best: &oS9e1pOGnOtxtQQe[.
Average: 0.05518796992481297
World Record: 0.3157894736842105
Best: &oS9e1pOGnOtxtQQe[.
Average: 0.058764302059497654
World Record: 0.3157894736842105
Best: &oS9e1pOGnOtxtQQe[.
Average: 0.06214736842105387
World Record: 0.3157894736842105
Best: &oS9e1pOGnOtxtQQe[.
Average: 0.06489278752436782
World Record: 0.3157

World Record: 0.42105263157894735
Best: >, bGK r+aot[toD[eY
Average: 0.11013635124078389
World Record: 0.42105263157894735
Best: >, bGK r+aot[toD[eY
Average: 0.11034008097164753
World Record: 0.42105263157894735
Best: >, bGK r+aot[toD[eY
Average: 0.1105183008282001
World Record: 0.42105263157894735
Best: >, bGK r+aot[toD[eY
Average: 0.11066120074053216
World Record: 0.42105263157894735
Best: >, bGK r+aot[toD[eY
Average: 0.1108143493060882
World Record: 0.42105263157894735
Best: >, bGK r+aot[toD[eY
Average: 0.11091781177079332
World Record: 0.42105263157894735
Best: >, bGK r+aot[toD[eY
Average: 0.11098587933246443
World Record: 0.42105263157894735
Best: >, bGK r+aot[toD[eY
Average: 0.11116450546655182
World Record: 0.42105263157894735
Best: >, bGK r+aot[toD[eY
Average: 0.11124401913874282
World Record: 0.42105263157894735
Best: >, bGK r+aot[toD[eY
Average: 0.11138687952106431
World Record: 0.42105263157894735
Best: >, bGK r+aot[toD[eY
Average: 0.11148752162094028
World Record: 0.4210526

KeyboardInterrupt: 

In [502]:
pop.get_generations()

461

In [516]:
len(pop.population)

24900

# Debugging

In [ ]:
draw(pop, 5, True)

In [ ]:
for i in range(len(pop.population)):
    print(pop.population[i].fitness, pop.population[i].get_phrase())



In [ ]:
pop.mating_pool

In [ ]:
for i in range(len(pop.population)):
    print(pop.population[i].fitness, pop.population[i].get_phrase())


In [ ]:

pop.mating_pool